In [7]:
import pandas as pd
from neo4j_parallel_spark_loader import monopartite

from benchmarking.utils.spark import create_spark_session

In [8]:
spark_session = create_spark_session()

Ivy Default Cache set to: /Users/alexandergilmore/.ivy2/cache
The jars for the packages stored in: /Users/alexandergilmore/.ivy2/jars
org.neo4j#neo4j-connector-apache-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-616d1d0b-5d0d-41df-b22e-655bec560348;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/alexandergilmore/Documents/projects/neo4j-parallel-spark-loader/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.neo4j#neo4j-connector-apache-spark_2.12;5.1.0_for_spark_3 in central
	found org.neo4j#neo4j-connector-apache-spark_2.12_common;5.1.0 in central
	found org.neo4j.driver#neo4j-java-driver;4.4.12 in central
	found org.reactivestreams#reactive-streams;1.0.4 in local-m2-cache
	found org.apache.xbean#xbean-asm6-shaded;4.10 in central
	found org.neo4j#neo4j-cypher-dsl;2022.9.0 in central
	found org.apiguardian#apiguardian-api;1.1.2 in local-m2-cache
:: resolution report :: resolve 144ms :: artifacts dl 6ms
	:: modules in use:
	org.apache.xbean#xbean-asm6-shaded;4.10 from central in [default]
	org.apiguardian#apiguardian-api;1.1.2 from local-m2-cache in [default]
	org.neo4j#neo4j-connector-apache-spark_2.12;5.1.0_for_spark_3 from central in [default]
	org.neo4j#neo4j-connector-apache-spark_2.12_common;5.1.0 from central in [default]
	org.neo4j#neo4j-cypher-dsl;2022.9.0 from central in [default]
	org.neo4j.driver#neo4j-java-driver;4.4.12 from central in [default]
	org.reactivestreams

In [9]:
# df = pd.read_csv("data/monopartite_data.csv")
df = spark_session.read.option("header", True).csv("data/monopartite_data.csv").drop_duplicates()

In [10]:
group_and_batch_df = monopartite.group_and_batch_spark_dataframe(df, "source", "target", 2)

In [11]:
group_and_batch_df.show()

+------+------+-------+-----+
|source|target|  group|batch|
+------+------+-------+-----+
| 81701| 97415|1 --> 0|    1|
| 88400| 97394|1 --> 0|    1|
| 84293| 28891|1 --> 0|    1|
| 70745| 89229|1 --> 0|    1|
| 84916| 52066|1 --> 0|    1|
| 55952| 81848|1 --> 0|    1|
| 52618| 73183|1 --> 0|    1|
| 71163| 39193|1 --> 0|    1|
| 70637| 93852|1 --> 1|    0|
| 12626| 65960|1 --> 1|    0|
| 76256| 61327|0 --> 1|    1|
| 54721| 72127|0 --> 1|    1|
| 18296| 98682|0 --> 1|    1|
| 56918| 64645|0 --> 1|    1|
| 91904| 10715|0 --> 1|    1|
| 54269| 72396|0 --> 1|    1|
| 65202| 13815|0 --> 0|    0|
| 93007| 95821|0 --> 0|    0|
| 80390| 77535|0 --> 0|    0|
| 84911| 72255|0 --> 0|    0|
+------+------+-------+-----+
only showing top 20 rows



In [24]:
test = group_and_batch_df.select("target", "group", "batch").drop_duplicates().groupBy("target", "batch").count().orderBy("count", ascending=False)

In [25]:
test.show()

+------+-----+-----+
|target|batch|count|
+------+-----+-----+
|  8270|    0|    1|
| 75630|    1|    1|
| 82293|    1|    1|
| 53468|    1|    1|
| 44297|    0|    1|
| 59665|    1|    1|
| 41738|    1|    1|
| 22706|    1|    1|
| 79800|    0|    1|
| 28480|    0|    1|
| 76211|    1|    1|
| 96548|    0|    1|
| 49266|    0|    1|
| 83080|    1|    1|
| 20550|    0|    1|
| 53033|    0|    1|
| 59770|    1|    1|
| 59774|    0|    1|
| 52790|    0|    1|
| 75544|    0|    1|
+------+-----+-----+
only showing top 20 rows



In [26]:
stack = group_and_batch_df.select("source", "group", "batch").withColumnRenamed("source", "source_or_target").union(group_and_batch_df.select("target", "group", "batch").withColumnRenamed("target", "source_or_target"))

In [28]:
stack_grouped = stack.select("source_or_target", "group", "batch").drop_duplicates().groupBy("source_or_target", "batch").count().orderBy("count", ascending=False)

In [29]:
stack_grouped.show()

+----------------+-----+-----+
|source_or_target|batch|count|
+----------------+-----+-----+
|           22811|    1|    2|
|           47772|    1|    2|
|           72959|    1|    2|
|           98492|    1|    2|
|           55936|    1|    2|
|           56770|    1|    2|
|           98773|    1|    2|
|           50573|    1|    2|
|           83723|    1|    2|
|           38282|    1|    2|
|           25763|    1|    2|
|           99926|    1|    2|
|           88881|    1|    2|
|           89668|    1|    2|
|           95252|    1|    2|
|           36928|    1|    2|
|           52622|    1|    2|
|           44548|    1|    2|
|           18474|    1|    2|
|           55848|    1|    2|
+----------------+-----+-----+
only showing top 20 rows



In [31]:
group_and_batch_df.select("*").where("source = 22811 or target = 22811").show()

+------+------+-------+-----+
|source|target|  group|batch|
+------+------+-------+-----+
| 22811| 61140|1 --> 0|    1|
| 97378| 22811|0 --> 1|    1|
+------+------+-------+-----+

